# Classical Explicit & Quasi-Explicit Arsenal

**Theme: The Rise and Fall of the Specific Solution**

## 1. Introduction

The mathematical journey begins with the optimistic 19th-century pursuit of closed-form solutions. This era of differential equations was characterized by a specific philosophical belief: that physical reality could be captured by explicit formulas—finite combinations of elementary functions and integrals.

In this notebook, we explore the "Exact Methods," a toolkit of clever substitutions and geometric insights designed to reduce differential equations to **quadrature** (direct integration). We will build the classical arsenal of special functions and integral transforms used to solve linear systems. However, as we push these methods to their limits, we will uncover the generic emergence of chaos, the breakdown of naive series expansions, and the necessity of "renormalization" to tame divergent solutions.



## 2. The Geometry of Quadrature (Exact Methods)

We commence with first-order ordinary differential equations (ODEs), seeking reductions that yield explicit integrals. The most fundamental of these is **reduction to quadrature**, where a differential equation is transformed into an expression where variables are separated.

### 2.1 Separable Equations & Exact Differentials

If the local geometry of the vector field allows for a decomposition of coordinates, we solve by direct integration. When separability is not immediate, we look for **Exact Differentials**. Consider the differential form:

$$M(x,y)dx + N(x,y)dy = 0$$

If the condition $\frac{\partial M}{\partial y} = \frac{\partial N}{\partial x}$ holds, the vector field is conservative (curl-free). This implies the existence of a potential function $\phi$ such that $M = \frac{\partial \phi}{\partial x}$ and $N = \frac{\partial \phi}{\partial y}$. Geometrically, solving the ODE corresponds to finding the level sets (contours) of this potential surface.



In [1]:
import sys
sys.path.insert(0, '../src')

import numpy as np
import plotly.graph_objects as go

from diffeq import PlotManager
from diffeq.widgets.base import InteractiveWidget, ParameterSlider
from diffeq.core.solvers import ODESolver

# [Interactive Vector Field. Users toggle between an Exact field (conservative) and a Non-Exact field (with curl), visualizing the existence (or lack) of a potential surface.]

class VectorFieldWidget(InteractiveWidget):
    def _setup_widgets(self):
        self.sliders['field_type'] = ParameterSlider(
            name='field_type',
            value=0.0,
            min_val=0.0,
            max_val=1.0,
            step=1.0,
            description='Field Type (0 = Exact, 1 = Non-Exact)'
        )

    def _update_plot(self, change=None):
        with self.output:
            self.output.clear_output(wait=True)
            params = self.get_parameters()
            is_exact = params['field_type'] < 0.5

            # Create grid with higher resolution for better vector field visualization
            x = np.linspace(-3, 3, 35)
            y = np.linspace(-3, 3, 35)
            X, Y = np.meshgrid(x, y)

            # Compute vector field
            if is_exact:
                # Conservative field: F = grad(phi), phi = (x² + y²)/2 → F = (x, y)
                U = X
                V = Y
                title = 'Exact (Conservative) Vector Field'
            else:
                # Non-conservative field: rotational, curl F ≠ 0
                U = -Y
                V = X
                title = 'Non-Exact (Rotational) Vector Field'

            # Create Plotly figure with vector field
            # Use skip=1 to show all arrows for full density
            skip = 1
            x_sparse = x[::skip]
            y_sparse = y[::skip]
            X_sparse, Y_sparse = np.meshgrid(x_sparse, y_sparse)
            U_sparse = U[::skip, ::skip]
            V_sparse = V[::skip, ::skip]

            # Create vectors using line traces (more efficient than annotations)
            fig = self.plot_manager.create_plotly_figure(width=800, height=700)

            # Normalize vectors for consistent arrow size (adjust scale for higher density)
            max_norm = np.max(np.sqrt(U_sparse**2 + V_sparse**2))
            scale = 0.25 / max_norm if max_norm > 0 else 0.25  # Smaller scale for denser field

            # Create arrow traces
            arrow_x = []
            arrow_y = []
            for i in range(U_sparse.shape[0]):
                for j in range(U_sparse.shape[1]):
                    xi, yi = X_sparse[i, j], Y_sparse[i, j]
                    ui, vi = U_sparse[i, j] * scale, V_sparse[i, j] * scale

                    # Arrow line
                    arrow_x.extend([xi, xi + ui, None])
                    arrow_y.extend([yi, yi + vi, None])

            fig.add_trace(
                go.Scatter(
                    x=arrow_x,
                    y=arrow_y,
                    mode='lines',
                    line=dict(color=self.plot_manager.config.colors[0], width=2),
                    showlegend=False,
                    hoverinfo='skip'
                )
            )

            # Add arrowheads (small triangles)
            head_x = []
            head_y = []
            for i in range(U_sparse.shape[0]):
                for j in range(U_sparse.shape[1]):
                    xi, yi = X_sparse[i, j], Y_sparse[i, j]
                    ui, vi = U_sparse[i, j] * scale, V_sparse[i, j] * scale
                    head_size = 0.06  # Slightly smaller arrowhead for denser field

                    # Perpendicular direction for arrowhead
                    perp = np.array([-vi, ui])
                    perp = perp / (np.linalg.norm(perp) + 1e-10) * head_size

                    tip_x, tip_y = xi + ui, yi + vi
                    head_x.extend([tip_x - ui*0.2 + perp[0], tip_x, tip_x - ui*0.2 - perp[0], tip_x - ui*0.2 + perp[0], None])
                    head_y.extend([tip_y - vi*0.2 + perp[1], tip_y, tip_y - vi*0.2 - perp[1], tip_y - vi*0.2 + perp[1], None])

            fig.add_trace(
                go.Scatter(
                    x=head_x,
                    y=head_y,
                    mode='lines',
                    fill='toself',
                    fillcolor=self.plot_manager.config.colors[0],
                    line=dict(color=self.plot_manager.config.colors[0], width=1),
                    showlegend=False,
                    hoverinfo='skip'
                )
            )

            fig.update_layout(
                title=title,
                xaxis_title='$x$',
                yaxis_title='$y$',
                xaxis=dict(scaleanchor="y", scaleratio=1, range=[-3.5, 3.5]),
                yaxis=dict(range=[-3.5, 3.5])
            )

            fig.show()

widget = VectorFieldWidget(title="Exact vs Non-Exact Vector Fields")
widget.display()


### 2.2 The Integrating Factor

When a linear equation $y' + P(x)y = Q(x)$ is not exact, we force exactness by introducing scalar curvature into the coordinates via an **Integrating Factor**, $\mu(x)$.

We seek $\mu$ such that the left-hand side becomes a total derivative:

$$\frac{d}{dx}[\mu(x)y] = \mu(x)y' + \mu'(x)y = \mu(x)[y' + P(x)y]$$

Matching terms implies $\mu'(x) = \mu(x)P(x)$, yielding the geometric correction term:

$$\mu(x) = \exp\left(\int P(x)dx\right)$$



### 2.3 Nonlinear Linearization: Bernoulli & Riccati

When linear structures fail, we turn to specific nonlinear forms that allow for linearization. The **Bernoulli Equation**, $y' + P(x)y = Q(x)y^n$, represents a nonlinear distortion of a linear system. It is linearized via the substitution $u = y^{1-n}$, revealing that the nonlinearity is merely an artifact of the coordinate choice.

More profound is the **Riccati Equation**, the prototypical model for quadratic nonlinearity:

$$y' = P(x) + Q(x)y + R(x)y^2$$

Historically analyzed as a curiosity (Riccati, 1724), this equation connects first-order nonlinearities to second-order linear systems. We employ the substitution:

$$y = -\frac{u'}{R(x)u}$$

Differentiating this ansatz and substituting it back into the Riccati equation creates a miraculous cancellation of the squared term $y^2$:

$$-\frac{u''}{R(x)u} + \frac{u'R'(x)}{R(x)^2u} + \frac{u'^2}{R(x)u^2} = P(x) - Q(x)\frac{u'}{R(x)u} - R(x)\frac{u'^2}{R(x)^2u^2}$$

This reduces the nonlinear problem to finding the null space of a linear differential operator.



In [2]:
# [Riccati Flow Visualization. A widget showing the solution y blowing up in finite time, while the underlying linear solution u remains smooth (passing through zero).]

import plotly.graph_objects as go
from plotly.subplots import make_subplots

class RiccatiWidget(InteractiveWidget):
    def _setup_widgets(self):
        self.sliders['t_max'] = ParameterSlider(
            name='t_max',
            value=4.5,
            min_val=0.5,
            max_val=6.0,
            step=0.1,
            description='Maximum Time $t_{\\max}$'
        )

    def _update_plot(self, change=None):
        with self.output:
            self.output.clear_output(wait=True)
            params = self.get_parameters()
            t_max = params['t_max']

            # Time array (dense near blow-up)
            t = np.linspace(0, t_max, 1000)

            # Riccati solution: y' = y² + 1, y(0)=0 → y(t) = tan(t)
            y = np.tan(t)
            y = np.clip(y, -100, 100)  # Prevent overflow in display

            # Corresponding linear solution u' = -u, u(0)=1 → u(t) = exp(-t)
            # Relation: y = -u'/u
            u = np.exp(-t)

            # Create Plotly subplots
            fig = make_subplots(
                rows=2, cols=1,
                subplot_titles=(
                    'Riccati Solution - Finite-Time Blow-Up',
                    'Underlying Linear Solution - Smooth & Decaying'
                ),
                vertical_spacing=0.12,
                row_heights=[0.5, 0.5]
            )

            # Apply dark theme
            fig.update_layout(
                template='plotly_dark',
                paper_bgcolor=self.plot_manager.config.background_color,
                plot_bgcolor=self.plot_manager.config.plot_bg_color,
                font=dict(
                    family=self.plot_manager.config.font_family,
                    size=self.plot_manager.config.font_size,
                    color=self.plot_manager.config.text_color
                ),
                height=800,
                width=1000,
                showlegend=True
            )

            # Top plot: Riccati solution
            fig.add_trace(
                go.Scatter(
                    x=t, y=y,
                    mode='lines',
                    name=r'$y(t) = \tan(t)$',
                    line=dict(color=self.plot_manager.config.colors[3], width=2)
                ),
                row=1, col=1
            )
            fig.add_vline(
                x=np.pi/2, line_dash="dash", line_color="gray",
                annotation_text="Blow-up at $t = \\pi/2$",
                row=1, col=1
            )

            # Bottom plot: Linear solution
            fig.add_trace(
                go.Scatter(
                    x=t, y=u,
                    mode='lines',
                    name=r'$u(t) = e^{-t}$',
                    line=dict(color=self.plot_manager.config.colors[1], width=2)
                ),
                row=2, col=1
            )
            fig.add_hline(y=0, line_dash="dash", line_color="gray", opacity=0.5, row=2, col=1)

            # Update axes
            fig.update_xaxes(title_text="$t$", row=2, col=1)
            fig.update_yaxes(title_text="$y(t)$", row=1, col=1)
            fig.update_yaxes(title_text="$u(t)$", row=2, col=1)

            fig.show()

widget = RiccatiWidget(title="Riccati Equation: Blow-Up vs Smooth Linear Solution")
widget.display()


## 3. The Limits of Exactness: Deterministic Chaos

The dream of universal integrability—that every system has a closed-form solution—shatters when we confront generic nonlinear systems. Even simple, deterministic systems can exhibit behavior that defies explicit analytical description.

Consider the **Lorenz System** (Lorenz, 1963), a simplified model of atmospheric convection:

$$\begin{align}
\frac{dx}{dt} &= \sigma(y - x) \\
\frac{dy}{dt} &= x(\rho - z) - y \\
\frac{dz}{dt} &= xy - \beta z
\end{align}$$

Despite being a system of polynomial ODEs, the solutions do not converge to stable equilibria or periodic limit cycles for certain parameter values (e.g., $\sigma=10$, $\beta=8/3$, $\rho=28$). Instead, they evolve on a **Strange Attractor**, a fractal set in phase space. This sensitivity to initial conditions implies that local existence theories (Picard-Lindstedt) do not guarantee global, elementary formulas.



In [3]:
# [The Chaos Slider. A 3D phase portrait of the Lorenz system. A slider for ρ transitions the system from a stable fixed point, to a limit cycle, to full chaos.]

from diffeq.widgets.base import InteractiveWidget, ParameterSlider
from diffeq.core.solvers import ODESolver
import plotly.graph_objects as go

class ChaosSliderWidget(InteractiveWidget):
    def _setup_widgets(self):
        self.sliders['rho'] = ParameterSlider(
            name='rho',
            value=28.0,
            min_val=0.0,
            max_val=50.0,
            step=0.5,
            description=r'$\rho$ (Rayleigh)',
            config=self.config
        )
        self.sigma = 10.0
        self.beta = 8.0/3.0

    def _update_plot(self, change=None):
        with self.output:
            self.output.clear_output(wait=True)
            params = self.get_parameters()
            rho = float(params['rho'])

            # Initial condition
            y0 = np.array([1.0, 1.0, 1.0])

            # Solve Lorenz system
            t_span = (0.0, 50.0)
            t, y = ODESolver.solve_lorenz(
                t_span=t_span,
                y0=y0,
                sigma=self.sigma,
                rho=rho,
                beta=self.beta,
                num_points=5000,
            )

            # Create Plotly 3D figure
            fig = go.Figure(data=[go.Scatter3d(
                x=y[:, 0],
                y=y[:, 1],
                z=y[:, 2],
                mode='lines',
                line=dict(
                    color=y[:, 0],  # Color by x-coordinate
                    colorscale='Viridis',
                    width=2
                ),
                name='Trajectory'
            )])

            fig.update_layout(
                template='plotly_dark',
                title=f'Lorenz Attractor ($\\rho = {rho:.1f}$)',
                scene=dict(
                    xaxis_title='$x$',
                    yaxis_title='$y$',
                    zaxis_title='$z$',
                    aspectmode='cube',
                    camera=dict(eye=dict(x=1.5, y=1.5, z=1.2))
                ),
                width=1000,
                height=800
            )

            fig.show()

widget = ChaosSliderWidget(title="Lorenz System: The Chaos Slider")
widget.display()


## 4. Special Functions & Spectral Geometry

When elementary functions ($\sin$, $\cos$, polynomials) fail to describe a system, we do not abandon exactness; rather, we expand our language. We define **Special Functions** as the canonical solutions to linear second-order ODEs arising from physical symmetries.

### 4.1 The Hypergeometric Ancestry

The unifying ancestor of many such functions is the **Hypergeometric Master Equation**:

$$z(1-z)w'' + [c - (a+b+1)z]w' - abw = 0$$

Through the **Frobenius Method**, we construct series solutions $w(z) = \sum_{n=0}^\infty a_n z^{n+r}$ near regular singular points. By merging singularities (taking confluent limits), we derive the vocabulary of mathematical physics:

* **Bessel Functions** (Cylindrical symmetry)
* **Legendre Polynomials** (Spherical symmetry)
* **Hermite Polynomials** (Quantum Harmonic Oscillator)

### 4.2 The Stokes Phenomenon

These series solutions exhibit complex behaviors in the complex plane. The **Stokes Phenomenon** (Stokes, 1857) describes how the asymptotic expansion of a function (like the Airy function, $\text{Ai}(z)$) changes discontinuously across specific rays called "Stokes lines," even though the function itself is analytic everywhere.



In [8]:
# [The Stokes Sector. A complex plane visualizer showing the magnitude of the Airy function. It highlights the "Stokes Lines" where the asymptotic behavior jumps from exponential decay to oscillation.]

from scipy.special import airy
import plotly.graph_objects as go

class AiryStokesWidget(InteractiveWidget):
    def _setup_widgets(self):
        self.sliders['x_range'] = ParameterSlider(
            name='x_range',
            value=6.0,
            min_val=2.0,
            max_val=10.0,
            step=0.5,
            description='Real Axis Range'
        )
        self.sliders['y_range'] = ParameterSlider(
            name='y_range',
            value=6.0,
            min_val=2.0,
            max_val=10.0,
            step=0.5,
            description='Imaginary Axis Range'
        )
        self.sliders['resolution'] = ParameterSlider(
            name='resolution',
            value=200,
            min_val=100,
            max_val=300,
            step=50,
            description='Resolution'
        )

    def _update_plot(self, change=None):
        with self.output:
            self.output.clear_output(wait=True)
            params = self.get_parameters()
            x_range_val = params['x_range']
            y_range_val = params['y_range']
            resolution = int(params['resolution'])

            x_range = (-x_range_val, x_range_val/2)
            y_range = (-y_range_val, y_range_val)

            # Create grid
            x = np.linspace(x_range[0], x_range[1], resolution)
            y = np.linspace(y_range[0], y_range[1], resolution)
            X, Y = np.meshgrid(x, y)
            Z = X + 1j * Y

            # Compute Airy function magnitude
            magnitude = np.zeros_like(Z, dtype=np.float64)
            for i in range(Z.shape[0]):
                for j in range(Z.shape[1]):
                    try:
                        ai, _, _, _ = airy(Z[i, j])
                        magnitude[i, j] = np.abs(ai)
                    except:
                        magnitude[i, j] = np.nan

            # Log scale for better visualization
            magnitude_log = np.log10(magnitude + 1e-10)

            # Create Plotly surface plot
            fig = go.Figure(data=[go.Surface(
                x=X,
                y=Y,
                z=magnitude_log,
                colorscale='Viridis',
                colorbar=dict(title="log₁₀(|Ai(z)|)"),
                hovertemplate='Re(z)=%{x:.2f}<br>Im(z)=%{y:.2f}<br>log|Ai(z)|=%{z:.2f}<extra></extra>'
            )])

            # Add Stokes lines as annotations (on the base plane)
            stokes_angles = [0, 2*np.pi/3, 4*np.pi/3]
            center_x, center_y = (x_range[0] + x_range[1])/2, (y_range[0] + y_range[1])/2
            max_r = np.sqrt((x_range[1] - x_range[0])**2 + (y_range[1] - y_range[0])**2) / 2

            for angle in stokes_angles:
                dx = max_r * np.cos(angle)
                dy = max_r * np.sin(angle)
                # Draw line on the base plane (z = min(magnitude_log))
                z_base = np.min(magnitude_log)
                fig.add_trace(go.Scatter3d(
                    x=[center_x - dx, center_x + dx],
                    y=[center_y - dy, center_y + dy],
                    z=[z_base, z_base],
                    mode='lines',
                    line=dict(color='red', width=4, dash='dash'),
                    name='Stokes Line',
                    showlegend=(angle == stokes_angles[0])
                ))

            fig.update_layout(
                template='plotly_dark',
                title='Airy Function Magnitude in Complex Plane (Stokes Lines)',
                scene=dict(
                    xaxis_title='Re(z)',
                    yaxis_title='Im(z)',
                    zaxis_title='log₁₀(|Ai(z)|)',
                    aspectmode='cube',
                    camera=dict(eye=dict(x=1.5, y=1.5, z=1.2))
                ),
                width=1000,
                height=800
            )

            fig.show()

widget = AiryStokesWidget(title="Airy Function: Stokes Phenomenon in Complex Plane")
widget.display()


### 4.3 Integral Transforms & Green's Functions

To handle linear complexity globally, we introduce **Integral Transforms**. The Laplace and Fourier transforms diagonalize differential operators, converting calculus into algebra. This leads to the method of **Green's Functions**, where the solution to an inhomogeneous equation $Lu = f$ is expressed as an integral kernel:

$$u(x) = \int G(x, \xi) f(\xi) d\xi$$

Here, $G(x, \xi)$ represents the impulse response of the system—the "physics" of the operator distilled into a single function.



## 5. Asymptotic Analysis & Perturbation Theory

When exact solutions are unavailable—as is the case for variable-coefficient problems or weak nonlinearities—we resort to approximation in limiting regimes ($\epsilon \to 0$ or $x \to \infty$).

### 5.1 WKB Approximation

For highly oscillatory systems or quantum mechanics, we use the **WKB Approximation** (Wentzel, Kramers, Brillouin). For an equation $\epsilon^2 y'' + Q(x)y = 0$ with $\epsilon \ll 1$, we propose the ansatz:

$$y(x) = A(x)e^{iS(x)/\epsilon}$$

Expanding $S$ in powers of $\epsilon$ yields the classical "eikonal" equation $(S')^2 = Q(x)$ and a transport equation for the amplitude. The solution typically takes the form:

$$y(x) \sim \frac{C}{\sqrt[4]{Q(x)}} \exp\left(\pm \frac{i}{\epsilon}\int^x \sqrt{Q(s)}ds\right)$$

This links the wave solution to classical ray trajectories.



In [5]:
# [WKB vs. Numerical. A plot showing a potential well Q(x). Users can toggle the WKB approximation overlay against the true numerical solution to see where the approximation holds and where it breaks (at turning points).]

from scipy.special import airy
import plotly.graph_objects as go

class WKBWidget(InteractiveWidget):
    def _setup_widgets(self):
        self.sliders['energy'] = ParameterSlider(
            name='energy',
            value=0.0,
            min_val=-5.0,
            max_val=5.0,
            step=0.1,
            description='Energy Level $E$'
        )

    def _update_plot(self, change=None):
        with self.output:
            self.output.clear_output(wait=True)
            params = self.get_parameters()
            E = float(params['energy'])

            # Potential V(x) = x (linear potential, Airy equation y'' = (x-E) y)
            # Standard form: y'' + Q(x) y = 0 with Q(x) = x - E
            x = np.linspace(-10, 10, 1000)
            Q = x - E

            # Exact solution: Airy function Ai(x - E) (decaying solution)
            Ai, _, _, _ = airy(x - E)
            y_exact = Ai

            # Normalize
            y_exact = y_exact / np.max(np.abs(y_exact)) if np.max(np.abs(y_exact)) > 0 else y_exact

            # Create Plotly figure
            fig = self.plot_manager.create_plotly_figure(width=1000, height=600)

            # Add exact solution
            fig.add_trace(
                go.Scatter(
                    x=x,
                    y=y_exact,
                    mode='lines',
                    name='Exact (Airy Ai)',
                    line=dict(color=self.plot_manager.config.colors[0], width=2)
                )
            )

            # Add turning point line
            fig.add_vline(
                x=E,
                line_dash="dot",
                line_color="gray",
                line_width=2,
                annotation_text=f'Turning Point $x = {E:.1f}$',
                annotation_position="top"
            )

            # Add zero line
            fig.add_hline(
                y=0,
                line_dash="dash",
                line_color="gray",
                line_width=1,
                opacity=0.3
            )

            # Add forbidden region shading (approximate with scatter fill)
            forbidden_mask = Q < 0
            if np.any(forbidden_mask):
                x_forbidden = x[forbidden_mask]
                y_forbidden_top = np.full_like(x_forbidden, 0.1)
                y_forbidden_bot = np.full_like(x_forbidden, -0.1)

                fig.add_trace(
                    go.Scatter(
                        x=np.concatenate([x_forbidden, x_forbidden[::-1]]),
                        y=np.concatenate([y_forbidden_top, y_forbidden_bot[::-1]]),
                        fill='toself',
                        fillcolor='rgba(255,0,0,0.2)',
                        line=dict(color='rgba(0,0,0,0)'),
                        name='Classically Forbidden',
                        showlegend=True
                    )
                )

            fig.update_layout(
                title=f'WKB vs Exact Solution (Linear Potential, Energy $E = {E:.1f}$)',
                xaxis_title='$x$',
                yaxis_title='$y(x)$',
                hovermode='x unified'
            )

            fig.show()

widget = WKBWidget(title="WKB Approximation vs Exact Airy Solution")
widget.display()


### 5.2 Poincaré-Lindstedt & Secular Terms

For nonlinear problems close to a solvable linear limit, we use **Perturbation Theory**. However, a naive expansion $x(t) = x_0(t) + \epsilon x_1(t) + \epsilon^2 x_2(t) + \cdots$ often fails due to **Secular Terms**—terms like $t\sin(t)$ that grow unbounded in time, violating physical boundedness.

To resolve this, we employ the **Poincaré-Lindstedt Method**. We introduce a "strained time" coordinate $\tau = \omega t$, where the frequency itself is expanded:

$$\omega = \omega_0 + \epsilon \omega_1 + \epsilon^2 \omega_2 + \cdots$$

By choosing specific values for the frequency corrections $\omega_n$, we can kill the resonant terms that cause secular growth.



In [6]:
# [The Duffing Oscillator. Animation showing the "Naive" perturbation solution diverging (secular growth) while the Poincaré-Lindstedt solution remains bounded and phase-locked with the true nonlinear solution.]

import plotly.graph_objects as go

class DuffingPerturbationWidget(InteractiveWidget):
    def _setup_widgets(self):
        self.sliders['epsilon'] = ParameterSlider(
            name='epsilon',
            value=0.3,
            min_val=0.0,
            max_val=1.0,
            step=0.05,
            description=r'$\epsilon$ (Nonlinearity Strength)'
        )

    def _update_plot(self, change=None):
        with self.output:
            self.output.clear_output(wait=True)
            params = self.get_parameters()
            eps = float(params['epsilon'])

            T = np.linspace(0, 50, 2000)

            # True numerical solution: d²x/dt² + x + εx³ = 0, x(0)=1, dx/dt(0)=0
            y0 = np.array([1.0, 0.0])
            sol = ODESolver.solve_duffing(T, y0, epsilon=eps)
            x_num = sol[:, 0]

            # Naive perturbation: x ≈ cos(t) + (ε/32)(cos(3t) - cos(t)) - (3ε t /8) sin(t)  ← secular term
            x_naive = np.cos(T) + (eps/32)*(np.cos(3*T) - np.cos(T)) - (3*eps*T/8)*np.sin(T)

            # Poincaré-Lindstedt: frequency correction ω ≈ 1 + (3ε/8), bounded periodic solution
            omega = 1 + (3*eps/8)
            tau = omega * T
            x_pl = np.cos(tau) + (eps/32)*(np.cos(3*tau) - np.cos(tau))

            # Create Plotly figure
            fig = self.plot_manager.create_plotly_figure(width=1000, height=600)

            # Add traces
            fig.add_trace(
                go.Scatter(
                    x=T,
                    y=x_num,
                    mode='lines',
                    name='Numerical (True)',
                    line=dict(color=self.plot_manager.config.colors[0], width=2)
                )
            )

            fig.add_trace(
                go.Scatter(
                    x=T,
                    y=x_naive,
                    mode='lines',
                    name='Naive Perturbation (Secular Growth)',
                    line=dict(color=self.plot_manager.config.colors[3], width=2, dash='dash')
                )
            )

            fig.add_trace(
                go.Scatter(
                    x=T,
                    y=x_pl,
                    mode='lines',
                    name='Poincaré-Lindstedt (Bounded)',
                    line=dict(color=self.plot_manager.config.colors[1], width=2, dash='dot')
                )
            )

            fig.update_layout(
                title=f'Duffing Oscillator Perturbation ($\\epsilon = {eps:.2f}$)',
                xaxis_title='Time $t$',
                yaxis_title='$x(t)$',
                xaxis=dict(range=[0, 50]),
                hovermode='x unified'
            )

            fig.show()

widget = DuffingPerturbationWidget(title="Duffing Oscillator: Secular Terms vs Poincaré-Lindstedt")
widget.display()


## 6. Renormalization & Resummation

Finally, we confront a dark secret of analysis: most perturbative series in physics are **asymptotic series** with zero radius of convergence (coefficients often grow as $n!$). To extract physical numbers from these divergences, we turn to **Resummation**.

### 6.1 Borel Summation

**Borel Summation** allows us to sum factorial divergent series. Given a divergent series $S(x) = \sum_{n=0}^\infty a_n x^n$, we define the Borel Transform:

$$B(t) = \sum_{n=0}^\infty \frac{a_n}{n!} t^n$$

If this new series converges, the "sum" of the original divergent series is defined as the Laplace transform of $B$:

$$S(x) = \int_0^\infty e^{-t} B(xt) dt$$

This technique is the mathematical engine behind the **Renormalization Group**, which tames infinities in Quantum Field Theory by systematically handling the scaling of parameters.



In [7]:
# [Borel Summation Widget. Interactive summation of the Euler series ∑ (-1)^n n! x^n. Display the wild divergence of partial sums versus the stable, finite value produced by Borel resummation.]

from scipy.integrate import quad
from scipy.special import exp1
import plotly.graph_objects as go

class BorelSummationWidget(InteractiveWidget):
    def _setup_widgets(self):
        self.sliders['x'] = ParameterSlider(
            name='x',
            value=0.5,
            min_val=0.01,
            max_val=1.5,
            step=0.01,
            description='Parameter $x > 0$'
        )
        self.sliders['N'] = ParameterSlider(
            name='N',
            value=20,
            min_val=5,
            max_val=40,
            step=1,
            description='Partial Sum Order $N$'
        )

    def _update_plot(self, change=None):
        with self.output:
            self.output.clear_output(wait=True)
            params = self.get_parameters()
            x_val = float(params['x'])
            N = int(params['N'])

            # Partial sums of divergent series S(x) = ∑_{n=0}^N (-1)^n n! x^n
            # Use logarithms to avoid factorial overflow
            n = np.arange(0, N+1, dtype=np.float64)
            partial_sums = np.zeros(N+1, dtype=np.float64)

            current_sum = 0.0
            for k in range(N+1):
                # Compute log(|term|) = log(n!) + n*log(x)
                # Use Stirling's approximation for large n
                if k < 20:
                    log_fact = np.sum(np.log(np.arange(1, k+1, dtype=np.float64)))
                else:
                    log_fact = k * np.log(k) - k + 0.5 * np.log(2 * np.pi * k)

                log_term = log_fact + k * np.log(x_val)
                term = (-1)**k * np.exp(log_term)
                current_sum += term
                partial_sums[k] = current_sum

            # Borel resummation: For Euler series, S(x) = ∫_0^∞ e^{-t} / (1 + xt) dt
            # This equals e^{1/x} * Ei(-1/x) / x where Ei is the exponential integral
            try:
                if x_val > 0:
                    # Use exponential integral for exact value
                    borel_sum = np.exp(1.0/x_val) * exp1(1.0/x_val) / x_val
                else:
                    borel_sum = np.nan
            except:
                # Fallback to numerical integration
                def borel_integrand(t):
                    if x_val * t < 50:  # Reasonable cutoff
                        return np.exp(-t) / (1.0 + x_val * t)
                    return 0.0

                try:
                    borel_sum, _ = quad(borel_integrand, 0, 100, limit=500)
                except:
                    borel_sum = np.nan

            # Create Plotly figure
            fig = self.plot_manager.create_plotly_figure(
                width=1000,
                height=600
            )

            # Add partial sums trace
            fig.add_trace(
                go.Scatter(
                    x=n,
                    y=partial_sums,
                    mode='lines+markers',
                    name='Partial Sums (Divergent)',
                    line=dict(
                        color=self.plot_manager.config.colors[0],
                        width=1.5
                    ),
                    marker=dict(size=4)
                )
            )

            # Add Borel sum line if valid
            if not np.isnan(borel_sum) and np.isfinite(borel_sum):
                fig.add_hline(
                    y=borel_sum,
                    line_dash="dash",
                    line_color=self.plot_manager.config.colors[3],
                    line_width=2,
                    annotation_text=f'Borel Sum ≈ {borel_sum:.4f}',
                    annotation_position="right"
                )

            # Update layout
            fig.update_layout(
                title=f'Euler Series at $x = {x_val:.2f}$: Divergence vs Borel Resummation',
                xaxis_title='Order $n$',
                yaxis_title='Partial Sum Value',
                hovermode='x unified'
            )

            fig.show()

widget = BorelSummationWidget(title="Borel Summation of the Divergent Euler Series")
widget.display()


---

### References

* **Borel, É.** (1899). *Mémoire sur les séries divergentes*. Annales Scientifiques de l'École Normale Supérieure.

* **Green, G.** (1828). *An essay on the application of mathematical analysis to the theories of electricity and magnetism*.

* **Lorenz, E. N.** (1963). "Deterministic nonperiodic flow". *Journal of the Atmospheric Sciences*.

* **Poincaré, H.** (1892). *Les méthodes nouvelles de la mécanique céleste*.

* **Riccati, J.** (1724). *Animadversiones in aequationes differentiales secundi gradus*. Acta Eruditorum.

* **Stokes, G. G.** (1857). "On the discontinuity of arbitrary constants which appear in divergent developments". *Transactions of the Cambridge Philosophical Society*.

* **Wentzel, G., Kramers, H. A., & Brillouin, L.** (1926). (Independent papers on the WKB approximation).

